<a href="https://colab.research.google.com/github/RichardTesla/datal/blob/main/Analyzing_Volatility_with_Advanced_Option_Pricing_Models_Heston_vs_Merton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Group Work Project III**

|  |  |
|:---|:---|
|**Name** |  Carolina Magda Roma, Siddeshkanth Logonathan, Chizaram Onukwufor  |
|**Prior Knowledge** |Derivative pricing, Heston model, Merton model, Monte Carlo simulation, Python    |
|**Keywords** |Put option, call option, european x american option, jumps, simulation  |

## **Step 1**

Q5.

In [ ]:
#@title #
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as ss

In [ ]:
#@title ##### Functions for Heston's model

### Stochastic volatility
def SDE_vol(v0, kappa, theta, sigma, T, M, Ite, rand, row, cho_matrix):
    dt = T / M  # T = maturity, M = number of time steps
    v = np.zeros((M + 1, Ite), dtype=np.float64)
    v[0] = v0
    sdt = np.sqrt(dt)  # Sqrt of dt
    for t in range(1, M + 1):
        ran = np.dot(cho_matrix, rand[:, t])
        v[t] = np.maximum(
            0,
            v[t - 1]
            + kappa * (theta - v[t - 1]) * dt
            + np.sqrt(v[t - 1]) * sigma * ran[row] * sdt,
        )
    return v

### Stochastic volatility equation for stock prices
def Heston_paths(S0, r, v, row, cho_matrix):
    S = np.zeros((M + 1, Ite), dtype=float)
    S[0] = S0
    sdt = np.sqrt(dt)
    for t in range(1, M + 1, 1):
        ran = np.dot(cho_matrix, rand[:, t])
        S[t] = S[t - 1] * np.exp((r - 0.5 * v[t - 1]) * dt + np.sqrt(v[t - 1]) * ran[row] * sdt)

    return S

### Generate random numbers
def random_number_gen(M, Ite):
    rand = np.random.standard_normal((2, M + 1, Ite))
    return rand

In [ ]:
#@title #### Implementation for correlation equal to -0.30
v0 = 0.032
kappa_v = 1.85
sigma_v = 0.35
theta_v = 0.045
rho = -0.30

S0 = 80  # Current underlying asset price
r = 0.055  # Risk-free rate
M0 = 90  # Number of time steps in a year - daily time steps 3 months = 90 days
T = 0.25  # Number of years
M = int(M0 * T)  # Total time steps
Ite = 1000000  # Number of simulations
dt = T / M  # Length of time step
K = 80

In [ ]:
#@title #### Generating random numbers from standard normal
rand = random_number_gen(M, Ite)


# Covariance Matrix
covariance_matrix = np.zeros((2, 2), dtype=np.float64)
covariance_matrix[0] = [1.0, rho]
covariance_matrix[1] = [rho, 1.0]
cho_matrix = np.linalg.cholesky(covariance_matrix)

In [ ]:
#@title ##### Volatility process paths
V = SDE_vol(v0, kappa_v, theta_v, sigma_v, T, M, Ite, rand, 1, cho_matrix)

# Underlying price process paths
S30 = Heston_paths(S0, r, V, 0, cho_matrix)

In [ ]:
#@title #### Heston Model to price european call and put options
def heston_call_mc(S, K, r, T, t):
    payoff = np.maximum(0, S[-1, :] - K)

    average = np.mean(payoff)

    return np.exp(-r * (T - t)) * average

def heston_put_mc(S, K, r, T, t):
    payoff = np.maximum(0, K - S[-1, :])

    average = np.mean(payoff)

    return np.exp(-r * (T - t)) * average

In [ ]:
#@title #
hcall30 = heston_call_mc(S30, K, r, T, 0)
print("European Call Price under Heston with a correlation equal to -0.30: ", round(hcall30,2))

hput30 = heston_put_mc(S30, K, r, T, 0)
print("European Put Price under Heston with a correlation equal to -0.30: ", round(hput30,2))

European Call Price under Heston with a correlation equal to -0.30:  3.47
European Put Price under Heston with a correlation equal to -0.30:  2.38


Q7a:

Considering correlation equal to -0.30

In [ ]:
#@title ###### Delta and Gamma are obtained via the finite difference method (by shocking the underlying price and delta, respectively, up and down)

def hcall_price_delta_gamma(S, K, r, T, t, dS):

  # Price
  hcall = heston_call_mc(S, K = K ,r = r, T = T, t=t)
  hput = heston_put_mc(S, K = K ,r = r, T = T, t=t)

  # Delta
  Sup = Heston_paths(S0+dS/2, r, V, 0, cho_matrix)
  Sdown = Heston_paths(S0-dS/2, r, V, 0, cho_matrix)

  eurocall_inc = heston_call_mc(Sup, K = K ,r = r, T = T, t=t)
  eurocall_dec = heston_call_mc(Sdown, K = K ,r = r, T = T, t=t)
  deltacall = (eurocall_inc - eurocall_dec) / dS

  europut_inc = heston_put_mc(Sup, K = K ,r = r, T = T, t=t)
  europut_dec = heston_put_mc(Sdown, K = K ,r = r, T = T, t=t)
  deltaput = (europut_inc - europut_dec) / dS

  # Gamma
  Supgu = Heston_paths((S0+dS/2)+dS/2, r, V, 0, cho_matrix)
  Supgd = Heston_paths((S0+dS/2)-dS/2, r, V, 0, cho_matrix)
  Sdgu = Heston_paths((S0-dS/2)+dS/2, r, V, 0, cho_matrix)
  Sdgd = Heston_paths((S0-dS/2)-dS/2, r, V, 0, cho_matrix)

  deltacallup = (heston_call_mc(Supgu, K = K ,r = r, T = T, t=t) - heston_call_mc(Supgd, K = K ,r = r, T = T, t=t)) / dS
  deltacalld = (heston_call_mc(Sdgu, K = K ,r = r, T = T, t=t) - heston_call_mc(Sdgd, K = K ,r = r, T = T, t=t)) / dS
  gammacall = (deltacallup - deltacalld) / dS

  deltaputup = (heston_put_mc(Supgu, K = K ,r = r, T = T, t=t) - heston_put_mc(Supgd, K = K ,r = r, T = T, t=t)) / dS
  deltaputd = (heston_put_mc(Sdgu, K = K ,r = r, T = T, t=t) - heston_put_mc(Sdgd, K = K ,r = r, T = T, t=t)) / dS
  gammaput = (deltaputup - deltaputd) / dS


  return hcall, hput, deltacall, deltaput, gammacall, gammaput

In [ ]:
#@title #
# Considering correlation equal to -0.30
dS = 1

# Option prices
hcall30 = hcall_price_delta_gamma(S30, K, r, T, 0, dS=dS)[0]
print("European Call Price under Heston with a correlation equal to -0.30: ", round(hcall30,2))

hput30 = hcall_price_delta_gamma(S30, K, r, T, 0, dS=dS)[1]
print("European Put Price under Heston with a correlation equal to -0.30: ", round(hput30,2))

# Deltas
delta_hcall30 = hcall_price_delta_gamma(S30, K, r, T, 0, dS=dS)[2]
print("European Call Delta under Heston with a correlation equal to -0.30: ", round(delta_hcall30,2))

delta_hput30 = hcall_price_delta_gamma(S30, K, r, T, 0, dS=dS)[3]
print("European Put Delta under Heston with a correlation equal to -0.30: ", round(delta_hput30,2))

# Gammas
gamma_hcall30 = hcall_price_delta_gamma(S30, K, r, T, 0, dS=dS)[4]
print("European Call Gamma under Heston with a correlation equal to -0.30: ", round(gamma_hcall30,2))

gamma_put30 = hcall_price_delta_gamma(S30, K, r, T, 0, dS=dS)[5]
print("European Put Gamma under Heston with a correlation equal to -0.30: ", round(gamma_put30,2))

European Call Price under Heston with a correlation equal to -0.30:  3.47
European Put Price under Heston with a correlation equal to -0.30:  2.38
European Call Delta under Heston with a correlation equal to -0.30:  0.6
European Put Delta under Heston with a correlation equal to -0.30:  -0.4
European Call Gamma under Heston with a correlation equal to -0.30:  0.05
European Put Gamma under Heston with a correlation equal to -0.30:  0.05


Q6.

In [ ]:
#@title #
v0 = 0.032
kappa_v = 1.85
sigma_v = 0.35
theta_v = 0.045
rho = -0.70

S0 = 80  # Current underlying asset price
r = 0.055  # Risk-free rate
M0 = 90  # Number of time steps in a year
T = 0.25  # Number of years
M = int(M0 * T)  # Total time steps
Ite = 1000000  # Number of simulations
dt = T / M  # Length of time step
K = 80

# Generating random numbers from standard normal
rand = random_number_gen(M, Ite)

# Covariance Matrix
covariance_matrix = np.zeros((2, 2), dtype=np.float64)
covariance_matrix[0] = [1.0, rho]
covariance_matrix[1] = [rho, 1.0]
cho_matrix = np.linalg.cholesky(covariance_matrix)

# Volatility process paths
V = SDE_vol(v0, kappa_v, theta_v, sigma_v, T, M, Ite, rand, 1, cho_matrix)

# Underlying price process paths
S = Heston_paths(S0, r, V, 0, cho_matrix)

In [ ]:
#@title #

hcall70 = heston_call_mc(S, K, r, T, 0)
print("European Call Price under Heston with a correlation equal to -0.70: ", round(hcall70,2) )

hput70 = heston_put_mc(S, K, r, T, 0)
print("European Put Price under Heston with a correlation equal to -0.70: ", round(hput70,2))

European Call Price under Heston with a correlation equal to -0.70:  3.5
European Put Price under Heston with a correlation equal to -0.70:  2.39


Q7b:

Considering correlation equal to -0.70

In [ ]:
#@title #

# Option prices
hcall70 = hcall_price_delta_gamma(S, K, r, T, 0, dS=dS)[0]
print("European Call Price under Heston with a correlation equal to -0.70: ", round(hcall70,2))

hput70 = hcall_price_delta_gamma(S, K, r, T, 0, dS=dS)[1]
print("European Put Price under Heston with a correlation equal to -0.70: ", round(hput70,2))

# Deltas
delta_hcall70 = hcall_price_delta_gamma(S, K, r, T, 0, dS=dS)[2]
print("European Call Delta under Heston with a correlation equal to -0.70: ", round(delta_hcall70,2))

delta_hput70 = hcall_price_delta_gamma(S, K, r, T, 0, dS=dS)[3]
print("European Put Delta under Heston with a correlation equal to -0.70: ", round(delta_hput70,2))

# Gammas
gamma_hcall70 = hcall_price_delta_gamma(S, K, r, T, 0, dS=dS)[4]
print("European Call Gamma under Heston with a correlation equal to -0.70: ", round(gamma_hcall70,2))

gamma_put70 = hcall_price_delta_gamma(S, K, r, T, 0, dS=dS)[5]
print("European Put Gamma under Heston with a correlation equal to -0.70: ", round(gamma_put70,2))

European Call Price under Heston with a correlation equal to -0.70:  3.5
European Put Price under Heston with a correlation equal to -0.70:  2.39
European Call Delta under Heston with a correlation equal to -0.70:  0.63
European Put Delta under Heston with a correlation equal to -0.70:  -0.37
European Call Gamma under Heston with a correlation equal to -0.70:  0.05
European Put Gamma under Heston with a correlation equal to -0.70:  0.05


## Team Member B. Jump Modeler

In [ ]:
#@title ### Importing packages

import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as ss

In [ ]:
#@title ### Variables Initialization

S0 = 80
K = 80
r = 0.055
sigma = 0.35
T = 3/12
t = 0
N = 90
M = 1000000
dt = T/N

mu = -0.5
delta = 0.22

In [ ]:
#@title ### Merton' stock price generator, option price calculator and greeks (delta, gamma)
def merton_stock_price(S0, sigma, T, N, M, dt, lamb):
    SM = np.zeros((N + 1, M))
    SM[0] = S0

    # rj
    rj = lamb * (np.exp(mu + 0.5 * delta**2) - 1)

    # Random numbers
    z1 = np.random.standard_normal((N + 1, M))
    z2 = np.random.standard_normal((N + 1, M))
    y = np.random.poisson(lamb * dt, (N + 1, M))

    for t in range(1, N + 1):
        SM[t] = SM[t - 1] * (
            np.exp((r - rj - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z1[t])
            + (np.exp(mu + delta * z2[t]) - 1) * y[t]
        )
        SM[t] = np.maximum(
            SM[t], 0.00001
        )  # To ensure that the price never goes below zero!

    return SM

def get_call_price_european(S, K):
    call_payoff = np.maximum(S[-1, :]-K, 0)
    call_price = np.mean(call_payoff)
    call_price = np.exp(-r * (T - t)) * call_price
    return call_price

def get_put_price_european(S, K):
    put_payoff = np.maximum(K-S[-1, :], 0)
    put_price = np.mean(put_payoff)
    put_price = np.exp(-r * (T - t)) * put_price
    return put_price

def approx_delta(S0, K, r, sigma, T, N, M, dt, lamb, dS, opt_type='call'):
    S_merton_inc = merton_stock_price(S0=S0+dS/2, sigma=sigma, T=T, N=N, M=M, lamb=lamb, dt=dt)
    S_merton_dec = merton_stock_price(S0=S0-dS/2, sigma=sigma, T=T, N=N, M=M, lamb=lamb, dt=dt)
    if opt_type == 'call':
      european_inc = get_call_price_european(S_merton_inc, K)
      european_dec = get_call_price_european(S_merton_dec, K)
    elif opt_type == 'put':
      european_inc = get_put_price_european(S_merton_inc, K)
      european_dec = get_put_price_european(S_merton_dec, K)
    delta = (european_inc - european_dec) / dS
    return delta

def approx_gamma(S0, K, r, sigma, T, N, M, dt, lamb, opt_type='call', dS=[2,1]):
    delta_up = approx_delta(S0, K, r, sigma, T, N, M, dt, lamb, dS=dS[0], opt_type=opt_type)
    delta_down = approx_delta(S0, K, r, sigma, T, N, M, dt, lamb, dS=dS[1], opt_type=opt_type)
    gamma = (delta_up-delta_down)/(dS[0]-dS[1])
    return gamma

## Q8.

In [ ]:
#@title ### setting jump intensity
lamb = 0.75

In [ ]:
#@title #### Generating Merton stock prices
S_merton = merton_stock_price(S0, sigma, T, N, M, dt, lamb)

In [ ]:
# @title #### Calculating Put and Call price

call_price = get_call_price_european(S_merton, K)
put_price = get_put_price_european(S_merton, K)
call_price = round(call_price, 2)
put_price = round(put_price, 2)

print(f'Call Price for the European Option based on a Merton Jump model with lambda: {lamb} is: ${call_price}')
print(f'Put Price for the European Option based on a Merton Jump model with lambda: {lamb} is: ${put_price}')

Call Price for the European Option based on a Merton Jump model with lambda: 0.75 is: $8.3
Put Price for the European Option based on a Merton Jump model with lambda: 0.75 is: $7.23


In [ ]:
#@title #### Calculating Delta value call and put
delta_call = approx_delta(S0=S0, K=K, r=r, sigma=sigma, T=T, N=N, M=M, dt=dt, dS=1, lamb=lamb, opt_type='call')
delta_call = round(delta_call, 2)
print(f'Delta for European Call option: {round(delta_call, 2)}')
delta_put = approx_delta(S0=S0, K=K, r=r, sigma=sigma, T=T, N=N, M=M, dt=dt, dS=1, lamb=lamb, opt_type='put')
delta_put = round(delta_put, 2)
print(f'Delta for European Put option: {round(delta_put, 2)}')


Delta for European Call option: 0.68
Delta for European Put option: -0.38


In [ ]:
#@title #### Calculating Gamma value
gamma_call = approx_gamma(S0=S0, K=K, r=r, sigma=sigma, T=T, N=N, M=M, dt=dt, lamb=lamb, opt_type='call')
gamma_call = round(gamma_call, 2)
print(f'Gamma for European Call option: {round(gamma_call, 2)}')
gamma_put = approx_gamma(S0=S0, K=K, r=r, sigma=sigma, T=T, N=N, M=M, dt=dt, lamb=lamb, opt_type='put')
gamma_put = round(gamma_put, 2)
print(f'Gamma for European Put option: {round(gamma_put, 2)}')

Gamma for European Call option: 0.02
Gamma for European Put option: 0.01


## Q9.

In [ ]:
#@title ### Setting Jump intensity
lamb = 0.25

In [ ]:
#@title ### Simulating stock prices
S_merton = merton_stock_price(S0, sigma, T, N, M, dt, lamb)

In [ ]:
#@title ### Pricing the European Options

call_price = get_call_price_european(S_merton, K)
put_price = get_put_price_european(S_merton, K)
call_price = round(call_price, 2)
put_price = round(put_price, 2)

print(f'Call Price for the European Option based on a Merton Jump model with lambda: {lamb} is: ${call_price}')
print(f'Put Price for the European Option based on a Merton Jump model with lambda: {lamb} is: ${put_price}')

Call Price for the European Option based on a Merton Jump model with lambda: 0.25 is: $6.82
Put Price for the European Option based on a Merton Jump model with lambda: 0.25 is: $5.75


In [ ]:
#@title #### Calculating Delta value call and put
delta_call = approx_delta(S0=S0, K=K, r=r, sigma=sigma, T=T, N=N, M=M, dt=dt, dS=1, lamb=lamb, opt_type='call')
delta_call = round(delta_call, 2)
print(f'Delta for European Call option: {round(delta_call, 2)}')
delta_put = approx_delta(S0=S0, K=K, r=r, sigma=sigma, T=T, N=N, M=M, dt=dt, dS=1, lamb=lamb, opt_type='put')
delta_put = round(delta_put, 2)
print(f'Delta for European Put option: {round(delta_put, 2)}')

Delta for European Call option: 0.61
Delta for European Put option: -0.41


In [ ]:
#@title #### Calculating Gamma value
gamma_call = approx_gamma(S0=S0, K=K, r=r, sigma=sigma, T=T, N=N, M=M, dt=dt, lamb=lamb, opt_type='call')
gamma_call = round(gamma_call, 2)
print(f'Gamma for European Call option: {round(gamma_call, 2)}')
gamma_put = approx_gamma(S0=S0, K=K, r=r, sigma=sigma, T=T, N=N, M=M, dt=dt, lamb=lamb, opt_type='put')
gamma_put = round(gamma_put, 2)
print(f'Gamma for European Put option: {round(gamma_put, 2)}')

Gamma for European Call option: -0.01
Gamma for European Put option: 0.01


Q10.

Considering a specific simulation, for Lambda value of 0.75, the Delta and Gamma of the European Call and Put option is the following:
  - Delta European Call: 0.65
  - Delta European Put: -0.35
  - Gamma European Call: 0.01
  - Gamma European Put: 0.01

For the Lambda value of 0.25, the Delta and Gamma of the following Call and Puts are:
  - Delta European Call: 0.59
  - Delta European Put: -0.41
  - Gamma European Call: 0.01
  - Gamma European Put: 0.02

  We can see that by reducing the Jump Intensity of the Merton model, the Delta of European Call reduced whereas the Delta of the European put increased. The Gamma values of the Call stayed the same whereas the Gamma value of the put increased by 0.01.


Please refer to **Table 1** for the final output based on current simulation.

### Team Member C. Model Validator

Q11.

In [ ]:
#@title ### Heston Model
print("European Call Price under Heston with a correlation equal to -0.30: ", round(hcall30,2))

print("European Put Price under Heston with a correlation equal to -0.30: ", round(hput30,2))

# Check put-call parity
print("Verifying Put-Call Parity:")
print("Call Price + S * exp(-r * T): ", hcall30 + S0 * np.exp(-r * T))
print("Put Price + S: ", hput30 + S0)
print("Put-Call Parity Test: ", round(hcall30 + S0 * np.exp(-r * T), 2) == round(hput30 + S0, 2))
put_call_parity_diff = hput30 - (hcall30 - S0 + K * np.exp(-r * T))
print("Put-Call Parity Difference:", put_call_parity_diff)

European Call Price under Heston with a correlation equal to -0.30:  3.47
European Put Price under Heston with a correlation equal to -0.30:  2.38
Verifying Put-Call Parity:
Call Price + S * exp(-r * T):  82.37467158006382
Put Price + S:  82.3804123038521
Put-Call Parity Test:  False
Put-Call Parity Difference: 0.005740723788282498


In [ ]:
#@title #
print("European Call Price under Heston with a correlation equal to -0.70: ", round(hcall70,2) )

print("European Put Price under Heston with a correlation equal to -0.70: ", round(hput70,2))
# Check put-call parity
print("Verifying Put-Call Parity:")
print("Call Price + S * exp(-r * T): ", hcall70 + S0 * np.exp(-r * T))
print("Put Price + S: ", hput70 + S0)
print("Put-Call Parity Test: ", round(hcall70 + S0 * np.exp(-r * T), 2) == round(hput70 + S0, 2))
put_call_parity_diff = hput70 - (hcall70 - S0 + K * np.exp(-r * T))
print("Put-Call Parity Difference:",  put_call_parity_diff)

European Call Price under Heston with a correlation equal to -0.70:  3.5
European Put Price under Heston with a correlation equal to -0.70:  2.39
Verifying Put-Call Parity:
Call Price + S * exp(-r * T):  82.40671520272645
Put Price + S:  82.39497274402027
Put-Call Parity Test:  False
Put-Call Parity Difference: -0.011742458706183179


It is important to note that although the test result is false for the Heston model, this depends on the simulation and the difference is very small.When running different simulations, we could confirm that the test result is true. Overall, the put-call parity **holds** for both model specifications.

In [ ]:
 #@title ### Merton Model
lamb = 0.75
call_price = get_call_price_european(S_merton, K)
put_price = get_put_price_european(S_merton, K)
call_price = round(call_price, 2)
put_price = round(put_price, 2)

print(f'Call Price for the European Option based on a Merton Jump model with lambda: {lamb} is: ${call_price}')
print(f'Put Price for the European Option based on a Merton Jump model with lambda: {lamb} is: ${put_price}')

# Check put-call parity
print("Verifying Put-Call Parity:")
print("Call Price + S * exp(-r * T): ", call_price + S0 * np.exp(-r * T))
print("Put Price + S: ", put_price + S0)
print("Put-Call Parity Test: ", round(call_price + S0 * np.exp(-r * T), 2) == round(put_price + S0, 2))
put_call_parity_diff = put_price - (call_price - S0 + K * np.exp(-r * T))
print("Put-Call Parity Difference:", put_call_parity_diff)

Call Price for the European Option based on a Merton Jump model with lambda: 0.75 is: $6.82
Put Price for the European Option based on a Merton Jump model with lambda: 0.75 is: $5.75
Verifying Put-Call Parity:
Call Price + S * exp(-r * T):  85.72752795736352
Put Price + S:  85.75
Put-Call Parity Test:  False
Put-Call Parity Difference: 0.022472042636479728


In [ ]:
#@title #
lamb = 0.25
call_price = get_call_price_european(S_merton, K)
put_price = get_put_price_european(S_merton, K)
call_price = round(call_price, 2)
put_price = round(put_price, 2)

print(f'Call Price for the European Option based on a Merton Jump model with lambda: {lamb} is: ${call_price}')
print(f'Put Price for the European Option based on a Merton Jump model with lambda: {lamb} is: ${put_price}')

# Check put-call parity
print("Verifying Put-Call Parity:")
print("Call Price + S * exp(-r * T): ", call_price + S0 * np.exp(-r * T))
print("Put Price + S: ", put_price + S0)
print("Put-Call Parity Test: ", round(call_price + S0 * np.exp(-r * T), 2) == round(put_price + S0, 2))
put_call_parity_diff = hput30 - (hcall30 - S0 + K * np.exp(-r * T))
print("Put-Call Parity Difference:", put_call_parity_diff)

Call Price for the European Option based on a Merton Jump model with lambda: 0.25 is: $6.82
Put Price for the European Option based on a Merton Jump model with lambda: 0.25 is: $5.75
Verifying Put-Call Parity:
Call Price + S * exp(-r * T):  85.72752795736352
Put Price + S:  85.75
Put-Call Parity Test:  False
Put-Call Parity Difference: 0.005740723788282498


Again, in general the put-call parity holds for european options as expected. The difference shown above is very small and depends on the simulation.

Q12.

Heston Model and Merton Model for 7 different strikes

In [ ]:
#@title ### Heston Model  for 7 different strikes
import numpy as np

import numpy as np

def heston_call_mc(S, K, r, T, t, v0, kappa_v, theta_v, sigma_v, rho, M, Ite):
    dt = (T - t) / M
    payoff = np.maximum(0, S[:, -1] - K)
    average = np.mean(payoff)
    return np.exp(-r * (T - t)) * average

# Set up parameters
S0 = 80  # Current underlying asset price
r = 0.055  # Risk-free rate
v0 = 0.032  # Initial variance
kappa_v = 1.85  # Mean reversion rate of variance
theta_v = 0.045  # Long-term variance
sigma_v = 0.35  # Volatility of variance
rho = -0.3  # Correlation between asset price and variance
M0 = 90  # Number of time steps in a year
T = 0.25  # Number of years
M = int(M0 * T)  # Total time steps
Ite = 1000000  # Number of simulations

moneyness_values = [0.85, 0.90, 0.95, 1, 1.05, 1.10, 1.15]
strikes = S0 / np.array(moneyness_values)

# Simulate the Heston Model paths
rand = np.random.standard_normal((2, M + 1, Ite))
V = SDE_vol(v0, kappa_v, theta_v, sigma_v, T, M, Ite, rand, 1, cho_matrix)
S = Heston_paths(S0, r, V, 0, cho_matrix)

# Run the Heston Model for different strikes
option_types = ['OTM', 'OTM', 'OTM', 'ATM', 'ITM', 'ITM', 'ITM']
for i, strike in enumerate(strikes):
    option_type = option_types[i]

    # Calculate option price
    if option_type == 'OTM':
        option_price = heston_call_mc(S, strike, r, T, 0, v0, kappa_v, theta_v, sigma_v, rho, M, Ite)
    elif option_type == 'ATM':
        option_price = heston_call_mc(S, strike, r, T, 0, v0, kappa_v, theta_v, sigma_v, rho, M, Ite)
    else:
        option_price = heston_call_mc(S, strike, r, T, 0, v0, kappa_v, theta_v, sigma_v, rho, M, Ite)

    # Print the results
    print(f"Strike: {strike:.2f}")
    print(f"Option Type: {option_type}")
    print(f"Option Price: {option_price:.2f}")
    print("---")


Strike: 94.12
Option Type: OTM
Option Price: 0.00
---
Strike: 88.89
Option Type: OTM
Option Price: 0.00
---
Strike: 84.21
Option Type: OTM
Option Price: 0.00
---
Strike: 80.00
Option Type: ATM
Option Price: 0.99
---
Strike: 76.19
Option Type: ITM
Option Price: 4.65
---
Strike: 72.73
Option Type: ITM
Option Price: 8.07
---
Strike: 69.57
Option Type: ITM
Option Price: 11.19
---


In [ ]:
#@title #
import numpy as np

def merton_stock_price(S0, sigma, T, N, M, dt, lamb):
    SM = np.zeros((N + 1, M))
    SM[0] = S0

    # Random numbers
    z1 = np.random.standard_normal((N + 1, M))
    z2 = np.random.standard_normal((N + 1, M))
    y = np.random.poisson(lamb * dt, (N + 1, M))

    for t in range(1, N + 1):
        SM[t] = SM[t - 1] * (
            np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z1[t])
            + (np.exp(mu + delta * z2[t]) - 1) * y[t]
        )
        SM[t] = np.maximum(
            SM[t], 0.00001
        )  # To ensure that the price never goes below zero!

    return SM

def get_call_price_european(S, K, r, T, t):
    call_payoff = np.maximum(S[-1, :] - K, 0)
    call_price = np.mean(call_payoff)
    call_price = np.exp(-r * (T - t)) * call_price
    return call_price

def get_put_price_european(S, K, r, T, t):
    put_payoff = np.maximum(K - S[-1, :], 0)
    put_price = np.mean(put_payoff)
    put_price = np.exp(-r * (T - t)) * put_price
    return put_price


# Set up parameters
S0 = 80  # Current underlying asset price
r = 0.055  # Risk-free rate
M0 = 90  # Number of time steps in a year
T = 0.25  # Number of years
M = int(M0 * T)  # Total time steps
N = 100  # Number of subintervals per time step
Ite = 1000000  # Number of simulations
dt = T / (M * N)  # Length of time step

moneyness_values = [0.85, 0.90, 0.95, 1, 1.05, 1.10, 1.15]
strikes = S0 / np.array(moneyness_values)

# Run the Merton Model for different strikes
option_types = ['OTM', 'OTM', 'OTM', 'ATM', 'ITM', 'ITM', 'ITM']
for i, strike in enumerate(strikes):
    option_type = option_types[i]

    # Calculate option price
    if option_type == 'OTM':
        option_price = get_call_price_european(S_merton, strike, r, T, 0)
    elif option_type == 'ATM':
        option_price = get_call_price_european(S_merton, strike, r, T, 0)
    else:
        option_price = get_call_price_european(S_merton, strike, r, T, 0)

    # Print the results
    print(f"Strike: {strike:.2f}")
    print(f"Option Type: {option_type}")
    print(f"Option Price: {option_price:.2f}")
    print("---")


Strike: 94.12
Option Type: OTM
Option Price: 2.00
---
Strike: 88.89
Option Type: OTM
Option Price: 3.28
---
Strike: 84.21
Option Type: OTM
Option Price: 4.90
---
Strike: 80.00
Option Type: ATM
Option Price: 6.82
---
Strike: 76.19
Option Type: ITM
Option Price: 8.96
---
Strike: 72.73
Option Type: ITM
Option Price: 11.24
---
Strike: 69.57
Option Type: ITM
Option Price: 13.56
---


# **Step 2**

Q13.




*   Considering the Heston model in Q5




In [ ]:
#@title #### Pricing an American Call option with Heston Model
def price_call_american_heston(S, K):
    C = np.zeros_like(S)
    call_payoff_exp = np.maximum(S[-1, :]-K, 0)
    C[-1, :] = call_payoff_exp
    discount_factor = np.exp(-r*dt)

    for i in range(S.shape[0]-2, -1, -1):
        call_payoff_value = np.maximum(S[i, :]-K, 0)
        call_holding_value = discount_factor*C[i+1, :]
        C[i,:] = np.maximum(call_payoff_value, call_holding_value)

    return C[0, :].mean(), C

In [ ]:
#@title #
hcall30_ame = price_call_american_heston(S30, 80)[0]
print("American Call Price with Heston model and correlation equal to -0.30: ", round(hcall30_ame, 2))


American Call Price with Heston model and correlation equal to -0.30:  5.48


*   Considering the Merton model in Q8

In [ ]:
#@title #### Pricing American Call option based on a Merton model stock price dynamic
def get_call_american(S, K):
    C = np.zeros_like(S)
    call_payoff_exp = np.maximum(S[-1, :]-K, 0)
    C[-1, :] = call_payoff_exp
    discount_factor = np.exp(-r*dt)

    for i in range(S.shape[0]-2, -1, -1):
        call_payoff_value = np.maximum(S[i, :]-K, 0)
        call_holding_value = discount_factor*C[i+1, :]
        C[i,:] = np.maximum(call_payoff_value, call_holding_value)

    return C[0, :].mean(), C

In [ ]:
#@title #### Pricing an American Call based on a Merton Model
S0 = 80
K = 80
r = 0.055
sigma = 0.35
T = 3/12
t = 0
N = 90
M = 1000000
dt = T/N

mu = -0.5
delta = 0.22

lamb = 0.75
S_merton = merton_stock_price(S0, sigma, T, N, M, dt, lamb)
call_price, _ = get_call_american(S_merton, K)
call_price = round(call_price, 2)

print(f'American Call value based on Merton model: ${call_price}')

American Call value based on Merton model: $10.65


We observe that the american call prices for both methods are higher than the european counterpart as the price dynamic incorporates stochastic volatility or jumps, respectively.

Q14.

Using a Heston model to price an European Up-and-in call option.

In [ ]:
#@title ##### Implementation for correlation equal to -0.70 (same of Q6)
v0 = 0.032
kappa_v = 1.85
sigma_v = 0.35
theta_v = 0.045
rho = -0.70

S0 = 95  # Current underlying asset price
B = 95
r = 0.055  # Risk-free rate
M0 = 90  # Number of time steps in a year - daily time steps 3 months = 90 days
T = 0.25  # Number of years
M = int(M0 * T)  # Total time steps
Ite = 1000000  # Number of simulations
dt = T / M  # Length of time step
K = 95

In [ ]:
#@title ##### Generating the stock price dynamic for the Heston Model

### Generating random numbers from standard normal
rand = random_number_gen(M, Ite)

### Covariance Matrix
covariance_matrix = np.zeros((2, 2), dtype=np.float64)
covariance_matrix[0] = [1.0, rho]
covariance_matrix[1] = [rho, 1.0]
cho_matrix = np.linalg.cholesky(covariance_matrix)

### Volatility process paths
V = SDE_vol(v0, kappa_v, theta_v, sigma_v, T, M, Ite, rand, 1, cho_matrix)

# Underlying price process paths
S = Heston_paths(S0, r, V, 0, cho_matrix)

In [ ]:
# @title ##### Pricing an Up-and-In (UAI) barrier european call option using Heston Model
def heston_call_uaimc(S, K, B, r, T, t):
    max_prices = np.amax(S, axis=0) >= B
    payoff = max_prices*np.maximum(0, S[-1, :] - K)

    average = np.mean(payoff)

    return np.exp(-r * (T - t)) * average

#### Heston Model to price european call
def heston_call_mc(S, K, r, T, t):
    payoff = np.maximum(0, S[-1, :] - K)

    average = np.mean(payoff)

    return np.exp(-r * (T - t)) * average

In [ ]:
#@title #
uaicall = heston_call_uaimc(S, K, B, r, T, 0)
print("European UAI Call Option Price with Heston Model stock price dyanmic and correlation equal to -0.70: ", round(uaicall, 2))

hcall70_new = heston_call_mc(S, K, r, T, 0)
print("European Call Option Price with Heston Model stock price dyanmic and correlation equal to -0.70: ", round(hcall70_new, 2))

European UAI Call Option Price with Heston Model stock price dyanmic and correlation equal to -0.70:  4.15
European Call Option Price with Heston Model stock price dyanmic and correlation equal to -0.70:  4.15


In general, the european UAI call option and the vanilla european call option result in the same price given that they are ATM and there is a probability that the underlying price will be higher than the barrier at any given point until maturity, therefore activating the option contract (even if the simulated prices end below the pre-defined barrier level). In this sense, the option can be exercised at the maturity date. With this characteristic, they result in same price.

Q15. Merton Model to price an European Down-and-In (DAI) put option.

In [ ]:
#@title #
B = 65
K = 65

S0 = 80
r = 0.055
sigma = 0.35
T = 3/12
t = 0
N = 90
M = 1000000
dt = T/N

mu = -0.5
delta = 0.22

In [ ]:
#@title #
lamb = 0.75
S_merton = merton_stock_price(S0, sigma, T, N, M, dt, lamb)

In [ ]:
#@title #### Functions to price simple European put option and a DAI European put option
def get_put_price_european(S, K):
    put_payoff = np.maximum(K-S[-1, :], 0)
    put_price = np.mean(put_payoff)
    put_price = np.exp(-r * (T - t)) * put_price
    return put_price

def get_put_price_european_DAI(S, K, B):
    put_price_expiration = S[-1, :]
    put_price_expiration[put_price_expiration < B] = 0
    put_payoff = np.maximum(K-put_price_expiration, 0)
    put_payoff = np.mean(put_payoff)
    put_price = np.exp(-r * (T - t)) * put_payoff
    return put_price

In [ ]:
#@title #### Simple and DAI European put price
simple_put_price = get_put_price_european(S_merton, K)
simple_put_price = round(simple_put_price, 2)
print(f'Simple European put price: ${simple_put_price}')

dai_european_put_price = get_put_price_european_DAI(S_merton, K, B)
dai_european_put_price = round(dai_european_put_price, 2)
print(f'DAI European put price: ${dai_european_put_price}')


Simple European put price: $3.53
DAI European put price: $15.78


In [ ]:
#@title ###### Additional test using input data from Q9.
lamb = 0.25
S_merton = merton_stock_price(S0, sigma, T, N, M, dt, lamb)

In [ ]:
#@title #### Simple and DAI European put price
simple_put_price = get_put_price_european(S_merton, K)
simple_put_price = round(simple_put_price, 2)
print(f'Simple European put price: ${simple_put_price}')

dai_european_put_price = get_put_price_european_DAI(S_merton, K, B)
dai_european_put_price = round(dai_european_put_price, 2)
print(f'DAI European put price: ${dai_european_put_price}')


Simple European put price: $1.57
DAI European put price: $10.5


For the lambda value of 0.75 and a given simulation, the price of of the DAI European put options are \$11.8 and for a simple European put option is \$2.77. The reason why the DAI put option is much more expensive is that the payoff kicks-in is only valid above the barrier level, thus limiting payoff and the stock price dynamic to stay withing the barrier level and the stock price at option writing. The simple put option is cheaper because the variability of the stock price is broader than the DAI option.

The same works for a lower lambda in the Jump process and pricing the DAI put option. The price of the DAI option is more expensive than a simple European option.